In [1]:
import tensorflow as tf 
import numpy as np 
from tensorflow import keras 
import os 
import math 
import random 
import pickle 
import matplotlib.pyplot as plt 
from collections import deque 

from vehicle_model_DDQN_notScaling import Environment 
from cell_model import CellModel 

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
drving_cycle = '../../OC_SIM_DB/OC_SIM_DB_Cycles/Highway/01_FTP72_fuds.mat'
battery_path = "../../OC_SIM_DB/OC_SIM_DB_Bat/OC_SIM_DB_Bat_nimh_6_240_panasonic_MY01_Prius.mat"
motor_path = "../../OC_SIM_DB/OC_SIM_DB_Mot/OC_SIM_DB_Mot_pm_95_145_X2.mat"
cell_model = CellModel()
env = Environment(cell_model, drving_cycle, battery_path, motor_path, 1)


In [3]:
# STATE_SIZE = env.calculation_comp["state_size"]
STATE_SIZE = 4
ACTION_SIZE = env.calculation_comp["action_size"] 
LEARNING_RATE = 0.00025 

TOTAL_EPISODES = 200
MAX_STEPS = 50000 

GAMMA = 0.95 

MAX_EPSILON = 1 
MIN_EPSILON = 0.01 
DECAY_RATE = 0.00002
BATCH_SIZE = 32 
TAU = 0.001 
DELAY_TRAINING = 1000 
EPSILON_MIN_ITER = 5000

In [4]:
primary_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])
target_network = keras.Sequential([
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#     keras.layers.BatchNormalization(), 
    keras.layers.Dense(ACTION_SIZE),
])

primary_network.compile(
    loss="mse", 
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
)

# for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
#     t.assign(p)

In [5]:
def update_network(primary_network, target_network): 
    for t, p in zip(target_network.trainable_variables, primary_network.trainable_variables): 
        t.assign(t * (1 - TAU) + p * TAU)

In [6]:
class Memory: 
    def __init__(self, max_memory): 
        self.max_memory = max_memory 
        self._samples = [] 
        
    def add_sample(self, sample): 
        self._samples.append(sample)
        if len(self._samples) > self.max_memory: 
            self._samples.pop(0)
        
    def sample(self, no_samples): 
        if no_samples > len(self._samples): 
            return random.sample(self._samples, len(self._samples))
        else: 
            return random.sample(self._samples, no_samples)
    
    @property
    def num_samples(self):
        return len(self._samples)
    

# memory = Memory(10000)

In [7]:
def choose_action(state, primary_network, eps): 
    if random.random() < eps: 
        return random.randint(0, ACTION_SIZE - 1)
    else: 
        return np.argmax(primary_network(np.array(state).reshape(1, -1))) 

In [8]:
def train(primary_network, target_network, memory): 
    batch = memory.sample(BATCH_SIZE)
    states = np.array([val[0] for val in batch]) 
    actions = np.array([val[1] for val in batch])
    rewards = np.array([val[2] for val in batch])
    next_states = np.array([np.zeros(STATE_SIZE) if val[3] is None else val[3]  
                            for val in batch])
    
    prim_qt = primary_network(states)
    prim_qtp1 = primary_network(next_states)
    target_q = prim_qt.numpy() 
    updates = rewards 
    valid_idxs = next_states.sum(axis=1) != 0 
    batch_idxs = np.arange(BATCH_SIZE)
    prim_action_tp1 = np.argmax(prim_qtp1.numpy(), axis=1)
    q_from_target = target_network(next_states)
    updates[valid_idxs] += GAMMA * q_from_target.numpy()[batch_idxs[valid_idxs], 
                                                        prim_action_tp1[valid_idxs]]
    
    target_q[batch_idxs, actions] = updates 
    loss = primary_network.train_on_batch(states, target_q)
    return loss 
    
    
    

In [9]:
def initialization_with_rewardFactor(reward_factor):
    env = Environment(cell_model, drving_cycle, battery_path, motor_path, reward_factor)
    
    memory = Memory(10000)
    
    primary_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(),  
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    target_network = keras.Sequential([
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()), 
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(30, activation="relu", kernel_initializer=keras.initializers.he_normal()),
#         keras.layers.BatchNormalization(), 
        keras.layers.Dense(ACTION_SIZE),
    ])
    primary_network.compile(
        loss="mse", 
        optimizer=keras.optimizers.Adam(lr=LEARNING_RATE) 
    )
    return env, memory, primary_network, target_network 
    

In [10]:
print("environment version: {}".format(env.version)) 

 
reward_factors = [10]
results_dict = {} 

for reward_factor in reward_factors: 
    eps = MAX_EPSILON 
    steps = 0
    episode_rewards = [] 
    episode_SOCs = [] 
    episode_FCs = [] 
    
    env, memory, primary_network, target_network = initialization_with_rewardFactor(reward_factor)
    for episode in range(TOTAL_EPISODES): 
        state = env.reset() 
        avg_loss = 0 
        total_reward = 0
        cnt = 1 

        while True:
            action = choose_action(state, primary_network, eps)
            next_state, reward, done = env.step(action)
            total_reward += reward 
            if done: 
                next_state = None 
            memory.add_sample((state, action, reward, next_state))

            if steps > DELAY_TRAINING: 
                loss = train(primary_network, target_network, memory)
                update_network(primary_network, target_network)
                eps = MIN_EPSILON + (MAX_EPSILON - MIN_EPSILON) * np.exp(-DECAY_RATE * steps)
            else: 
                loss = -1

            avg_loss += loss 
            steps += 1 

            if done: 
                if steps > DELAY_TRAINING: 
                    SOC_deviation_history = np.sum(np.abs(np.array(env.history["SOC"]) - 0.6)) 
                    avg_loss /= cnt 
                    print('Episode: {}'.format(episode + 1),
                          'Total reward: {}'.format(total_reward), 
                          'Explore P: {:.4f}'.format(eps), 
                          "SOC: {:.4f}".format(env.SOC), 
                         "Cumulative_SOC_deviation: {:.4f}".format(SOC_deviation_history), 
                         "Fuel Consumption: {:.4f}".format(env.fuel_consumption), 
                         )
                else: 
                    print(f"Pre-training...Episode: {i}")
                
                episode_rewards.append(total_reward)
                episode_SOCs.append(env.SOC)
                episode_FCs.append(env.fuel_consumption)
                break 

            state = next_state 
            cnt += 1 
    
    results_dict[reward_factor] = {
        "rewards": episode_rewards, 
        "SOCs": episode_SOCs, 
        "FCs": episode_FCs 
    }
            
    

environment version: 1

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constru

maximum steps, simulation is done ... 
Episode: 40 Total reward: -4685.998528035451 Explore P: 0.3401 SOC: 1.0000 Cumulative_SOC_deviation: 457.6123 Fuel Consumption: 109.8753
maximum steps, simulation is done ... 
Episode: 41 Total reward: -4712.573878267805 Explore P: 0.3311 SOC: 1.0000 Cumulative_SOC_deviation: 459.1653 Fuel Consumption: 120.9211
maximum steps, simulation is done ... 
Episode: 42 Total reward: -4743.962006191131 Explore P: 0.3224 SOC: 1.0000 Cumulative_SOC_deviation: 462.6257 Fuel Consumption: 117.7051
maximum steps, simulation is done ... 
Episode: 43 Total reward: -4847.147205339332 Explore P: 0.3140 SOC: 1.0000 Cumulative_SOC_deviation: 473.2140 Fuel Consumption: 115.0072
maximum steps, simulation is done ... 
Episode: 44 Total reward: -4781.799621872613 Explore P: 0.3057 SOC: 1.0000 Cumulative_SOC_deviation: 466.2941 Fuel Consumption: 118.8586
maximum steps, simulation is done ... 
Episode: 45 Total reward: -4721.931871608275 Explore P: 0.2977 SOC: 1.0000 Cumula

maximum steps, simulation is done ... 
Episode: 87 Total reward: -2434.389070211449 Explore P: 0.1008 SOC: 1.0000 Cumulative_SOC_deviation: 235.2392 Fuel Consumption: 81.9968
maximum steps, simulation is done ... 
Episode: 88 Total reward: -4299.855138909701 Explore P: 0.0983 SOC: 1.0000 Cumulative_SOC_deviation: 420.0547 Fuel Consumption: 99.3084
maximum steps, simulation is done ... 
Episode: 89 Total reward: -1603.3101956985877 Explore P: 0.0960 SOC: 1.0000 Cumulative_SOC_deviation: 152.8674 Fuel Consumption: 74.6361
maximum steps, simulation is done ... 
Episode: 90 Total reward: -3371.8161888655322 Explore P: 0.0936 SOC: 1.0000 Cumulative_SOC_deviation: 329.6682 Fuel Consumption: 75.1347
maximum steps, simulation is done ... 
Episode: 91 Total reward: -2679.864257221907 Explore P: 0.0914 SOC: 1.0000 Cumulative_SOC_deviation: 260.6919 Fuel Consumption: 72.9450
maximum steps, simulation is done ... 
Episode: 92 Total reward: -4362.27951627063 Explore P: 0.0892 SOC: 1.0000 Cumulative

maximum steps, simulation is done ... 
Episode: 134 Total reward: -4579.895142297271 Explore P: 0.0350 SOC: 1.0000 Cumulative_SOC_deviation: 448.0411 Fuel Consumption: 99.4844
maximum steps, simulation is done ... 
Episode: 135 Total reward: -3886.864486673853 Explore P: 0.0343 SOC: 1.0000 Cumulative_SOC_deviation: 380.3147 Fuel Consumption: 83.7172
maximum steps, simulation is done ... 
Episode: 136 Total reward: -4890.618142275732 Explore P: 0.0336 SOC: 1.0000 Cumulative_SOC_deviation: 477.1398 Fuel Consumption: 119.2203
maximum steps, simulation is done ... 
Episode: 137 Total reward: -4198.529785704102 Explore P: 0.0330 SOC: 1.0000 Cumulative_SOC_deviation: 409.6465 Fuel Consumption: 102.0650
maximum steps, simulation is done ... 
Episode: 138 Total reward: -4427.32915458687 Explore P: 0.0324 SOC: 1.0000 Cumulative_SOC_deviation: 431.8549 Fuel Consumption: 108.7805
maximum steps, simulation is done ... 
Episode: 139 Total reward: -4581.675133711028 Explore P: 0.0318 SOC: 1.0000 Cum

maximum steps, simulation is done ... 
Episode: 181 Total reward: -2814.4702262289234 Explore P: 0.0169 SOC: 1.0000 Cumulative_SOC_deviation: 273.0978 Fuel Consumption: 83.4923
maximum steps, simulation is done ... 
Episode: 182 Total reward: -405.128101721389 Explore P: 0.0167 SOC: 0.6209 Cumulative_SOC_deviation: 36.3082 Fuel Consumption: 42.0460
maximum steps, simulation is done ... 
Episode: 183 Total reward: -2401.5806641475515 Explore P: 0.0165 SOC: 0.4918 Cumulative_SOC_deviation: 236.7599 Fuel Consumption: 33.9816
maximum steps, simulation is done ... 
Episode: 184 Total reward: -1959.6628259063398 Explore P: 0.0163 SOC: 0.4802 Cumulative_SOC_deviation: 192.7206 Fuel Consumption: 32.4567
maximum steps, simulation is done ... 
Episode: 185 Total reward: -1316.7090188230102 Explore P: 0.0162 SOC: 0.5170 Cumulative_SOC_deviation: 128.2923 Fuel Consumption: 33.7861
maximum steps, simulation is done ... 
Episode: 186 Total reward: -1459.8030651208533 Explore P: 0.0160 SOC: 0.5126 Cu

In [11]:
# plt.figure(figsize=(20, 10))
# for size, history in results_dict.items(): 
#     plt.plot(history, label=size, linewidth=3.0) 


# plt.grid() 
# plt.legend(fontsize=20)
# plt.xlabel("episode number", fontsize=20) 
# plt.ylabel("total rewards", fontsize=20) 
# plt.xlim([0, 120])


# plt.savefig("replay_memory_size_effect_300.png")
with open("results_notScaling_2.pkl", "wb") as f: 
    pickle.dump(results_dict, f, pickle.HIGHEST_PROTOCOL)

In [12]:
# with open("results/replay_memory_size_effect.pkl", "rb") as f: 
#     data = pickle.load(f)
    
# data